# Deep Learning Kaggle Competition: Footprint Image Classification

- **Student Name:** [TO FILL]
- **Student ID:** [TO FILL]
- **Kaggle Username:** [TO FILL]
- **Final Private Leaderboard Score:** [TO FILL]
- **Total Number of Submissions:** [TO FILL]

# Section 1: Introduction

**Problem:** Sex classification from footprint images for forensic analysis

**Why Deep Learning:** CNNs automatically learn features, traditional ML requires manual engineering

**Objectives:** Baseline → experiments → SOTA → explainability → Kaggle

# Section 2: EDA & Preprocessing

This section covers:
- Setup and imports
- Data loading and exploration
- Dataset statistics and visualization
- Data preprocessing and augmentation

In [ ]:
# Setup & Imports
!pip install timm optuna -q

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import timm
import optuna
from optuna.trial import TrialState
from torchvision import transforms, models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
import random
import copy
from pathlib import Path
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

# Device selection
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

# Set random seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Initialize global variables (will be updated based on actual data)
INPUT_CHANNELS = 3  # Default to RGB
IS_GRAYSCALE = False
BATCH_SIZE = 128  # Balanced: good speed, safer than 512

print('Setup complete!')
print('\nNow run ONE of the following cells to set data paths:')
print('  - OPTION A: Local data (next cell)')
print('  - OPTION B: Google Drive data (cell after that)')

In [ ]:
# =============================================================================
# OPTION A: Local Data (Run this cell if running locally)
# =============================================================================
# Data paths for local execution
DATA_DIR = Path('./data')
TRAIN_DIR = Path('./data/train')
TEST_DIR = Path('./data/test')

print('Using LOCAL data paths:')
print(f'  DATA_DIR: {DATA_DIR}')
print(f'  TRAIN_DIR: {TRAIN_DIR}')
print(f'  TEST_DIR: {TEST_DIR}')

In [ ]:
# =============================================================================
# OPTION B: Google Drive Data (Run this cell if running in Google Colab)
# =============================================================================
# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# # Data paths for Google Colab - adjust if your folder structure differs
# DATA_DIR = Path('/content/drive/MyDrive/Colab Files/DeepLearningData')
# TRAIN_DIR = DATA_DIR / 'train'
# TEST_DIR = DATA_DIR / 'test'

# print('Using GOOGLE DRIVE data paths:')
# print(f'  DATA_DIR: {DATA_DIR}')
# print(f'  TRAIN_DIR: {TRAIN_DIR}')
# print(f'  TEST_DIR: {TEST_DIR}')

# # Verify paths exist
# if DATA_DIR.exists():
#     print('\nData directory found!')
#     print(f'Contents: {list(DATA_DIR.iterdir())}')
# else:
#     print(f'\nWARNING: Data directory not found at {DATA_DIR}')
    # print('Please check your Google Drive path.')

In [ ]:
# Load Training Data
def load_data(data_dir):
    """Load image paths and labels from directory structure"""
    paths, labels = [], []
    for lbl in [0, 1]:
        folder = data_dir / str(lbl)
        if folder.exists():
            for ext in ['*.jpg', '*.png']:
                for p in folder.glob(ext):
                    paths.append(str(p))
                    labels.append(lbl)
    return paths, labels

# Load training data
if TRAIN_DIR.exists():
    train_paths, train_labels = load_data(TRAIN_DIR)
    train_df = pd.DataFrame({'path': train_paths, 'label': train_labels})
    print(f'Loaded {len(train_df)} training images')
    print('\nClass distribution:')
    print(train_df['label'].value_counts().sort_index())
else:
    train_df = pd.DataFrame()
    print('Training data directory not found!')

In [ ]:
# Visualize Class Distribution
if len(train_df) > 0:
    plt.figure(figsize=(8, 4))
    train_df['label'].value_counts().sort_index().plot(kind='bar')
    plt.title('Class Distribution')
    plt.xlabel('Class (0=Female, 1=Male)')
    plt.ylabel('Count')
    plt.xticks(rotation=0)
    plt.show()

In [ ]:
# Detect Image Properties
if len(train_df) > 0:
    # Check first image to determine if grayscale or RGB
    sample_img = Image.open(train_df.iloc[0]['path'])
    IS_GRAYSCALE = sample_img.mode == 'L'
    INPUT_CHANNELS = 1 if IS_GRAYSCALE else 3
    
    print(f'Image mode: {sample_img.mode}')
    print(f'Input channels: {INPUT_CHANNELS} ({"Grayscale" if IS_GRAYSCALE else "RGB"})')
    print(f'Image size: {sample_img.size}')

In [ ]:
# Define Transforms
if len(train_df) > 0:
    # Normalization parameters
    mean_std = ([0.5], [0.5]) if IS_GRAYSCALE else ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

    # Training transforms with STRONG augmentation for better generalization
    train_transform = transforms.Compose([
        transforms.Resize((256, 256)),  # Resize larger for random crop
        transforms.RandomCrop(224),      # Random crop for position invariance
        transforms.RandomHorizontalFlip(p=0.5),  # Left/right flip
        transforms.RandomRotation(15),   # Rotation ±15 degrees
        transforms.ColorJitter(
            brightness=0.2,  # Random brightness adjustment
            contrast=0.2,    # Random contrast adjustment
            saturation=0.1,  # Random saturation adjustment
            hue=0.05         # Slight hue shift
        ),
        transforms.RandomAffine(
            degrees=0,              # No additional rotation
            translate=(0.1, 0.1),   # Shift up to 10% in x/y
            scale=(0.9, 1.1),       # Scale 90% to 110%
            shear=5                 # Shear up to 5 degrees
        ),
        transforms.ToTensor(),
        transforms.Normalize(*mean_std),
        transforms.RandomErasing(p=0.2, scale=(0.02, 0.15))  # Random patch erasing
    ])

    # Validation/test transforms (no augmentation - deterministic)
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(*mean_std)
    ])

    print('Transforms defined with strong augmentation:')
    print('  - RandomCrop (256->224)')
    print('  - RandomHorizontalFlip')
    print('  - RandomRotation (±15°)')
    print('  - ColorJitter (brightness, contrast, saturation, hue)')
    print('  - RandomAffine (translate, scale, shear)')
    print('  - RandomErasing (20% probability)')

In [ ]:
# Create Dataset Class and DataLoaders
class FootprintDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert('L' if IS_GRAYSCALE else 'RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.labels[idx]

if len(train_df) > 0:
    # Split into train and validation sets (80/20)
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        train_df['path'].tolist(),
        train_df['label'].tolist(),
        test_size=0.2,
        random_state=42,
        stratify=train_df['label']
    )
    
    # Create datasets
    train_dataset = FootprintDataset(train_paths, train_labels, train_transform)
    val_dataset = FootprintDataset(val_paths, val_labels, val_transform)
    
    # Create dataloaders (num_workers=0 for Windows compatibility, but larger batch size for better GPU utilization)
    train_loader = DataLoader(
        train_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=True, 
        num_workers=0,  # Windows-safe: single-threaded loading
        pin_memory=True  # Faster GPU transfer
    )
    val_loader = DataLoader(
        val_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False, 
        num_workers=0,  # Windows-safe: single-threaded loading
        pin_memory=True
    )
    
    print(f'Train samples: {len(train_dataset)}')
    print(f'Validation samples: {len(val_dataset)}')
    print(f'Batch size: {BATCH_SIZE}')
    print(f'Note: num_workers=0 for Windows compatibility, batch_size={BATCH_SIZE} for better GPU utilization')

# Section 3: Baseline Model

This section implements a simple CNN from scratch to establish baseline performance.

In [ ]:
# Define Baseline CNN Architecture
class BaselineCNN(nn.Module):
    def __init__(self, num_classes=2, input_channels=3):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(input_channels, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

print('Baseline CNN architecture defined')

In [ ]:
# Training and Evaluation Functions
def train_epoch(model, loader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in tqdm(loader, desc='Training', leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


def evaluate(model, loader, criterion, device):
    """Evaluate model on validation/test set"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc='Evaluating', leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc, all_preds, all_labels

print('Training functions defined')

In [ ]:
# Generic Training Loop Function
def train_model(model, train_loader, val_loader, config, device):
    """
    Generic training function that works with any model and config
    
    config should contain: epochs, lr, optimizer ('sgd' or 'adam'), weight_decay (optional)
    """
    criterion = nn.CrossEntropyLoss()
    
    # Create optimizer
    if config['optimizer'] == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=config['lr'], 
                             momentum=0.9, weight_decay=config.get('weight_decay', 0))
    else:  # adam
        optimizer = optim.Adam(model.parameters(), lr=config['lr'], 
                              weight_decay=config.get('weight_decay', 0))
    
    # Training history
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    best_val_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    
    # Training loop
    for epoch in range(config['epochs']):
        print(f"\nEpoch {epoch+1}/{config['epochs']}")
        
        # Train
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        
        # Validate
        val_loss, val_acc, _, _ = evaluate(model, val_loader, criterion, device)
        
        # Store history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
    
    # Load best weights
    model.load_state_dict(best_model_wts)
    print(f'\nBest validation accuracy: {best_val_acc:.4f}')
    
    return model, history, best_val_acc

print('Generic training loop defined')

In [ ]:
# Train Baseline Model
if len(train_df) > 0:
    # Configuration for baseline
    baseline_config = {
        'epochs': 10,
        'lr': 0.001,
        'optimizer': 'sgd'
    }
    
    # Create and train model
    baseline_model = BaselineCNN(num_classes=2, input_channels=INPUT_CHANNELS).to(device)
    baseline_model, baseline_hist, baseline_acc = train_model(
        baseline_model, train_loader, val_loader, baseline_config, device
    )
    
    # Initialize results tracking
    experiment_results = [{
        'name': 'Baseline',
        'val_accuracy': baseline_acc,
        'val_loss': baseline_hist['val_loss'][-1]
    }]
    
    print(f'\nBaseline model training complete!')
    print(f'Final validation accuracy: {baseline_acc:.4f}')

In [ ]:
# Plot Learning Curves
if len(train_df) > 0:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Loss curves
    ax1.plot(baseline_hist['train_loss'], 'o-', label='Train')
    ax1.plot(baseline_hist['val_loss'], 's-', label='Validation')
    ax1.set_title('Baseline Model Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True)
    
    # Accuracy curves
    ax2.plot(baseline_hist['train_acc'], 'o-', label='Train')
    ax2.plot(baseline_hist['val_acc'], 's-', label='Validation')
    ax2.set_title('Baseline Model Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

# Section 4: State-of-the-Art Model Analysis

This section analyzes three SOTA architectures for potential use in the task.

In [ ]:
# Helper function to count parameters
def count_params(model):
    """Count trainable parameters in a model"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print('Parameter counting function defined')

In [ ]:
# Model 1: ResNet-18
print('='*60)
print('1. ResNet-18')
print('='*60)

r18 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Modify first conv layer if grayscale
if INPUT_CHANNELS == 1:
    r18.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Replace final layer for binary classification
r18.fc = nn.Linear(r18.fc.in_features, 2)

print(f'Parameters: {count_params(r18):,}')
print(f'Input channels: {INPUT_CHANNELS}')
print(f'Output classes: 2')
print('\nKey features:')
print('- Residual connections (skip connections)')
print('- Deep architecture (18 layers)')
print('- Pre-trained on ImageNet')

In [ ]:
# Model 2: EfficientNet-B0
print('='*60)
print('2. EfficientNet-B0')
print('='*60)

eff = timm.create_model('efficientnet_b0', pretrained=True, num_classes=2, in_chans=INPUT_CHANNELS)

print(f'Parameters: {count_params(eff):,}')
print(f'Input channels: {INPUT_CHANNELS}')
print(f'Output classes: 2')
print('\nKey features:')
print('- Compound scaling (depth, width, resolution)')
print('- Mobile inverted bottleneck convolutions')
print('- Efficient architecture, fewer params than ResNet')
print('- Pre-trained on ImageNet')

In [ ]:
# Model 3: Vision Transformer (ViT)
print('='*60)
print('3. Vision Transformer (ViT-Base/16)')
print('='*60)

vit = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=2, in_chans=INPUT_CHANNELS)

print(f'Parameters: {count_params(vit):,}')
print(f'Input channels: {INPUT_CHANNELS}')
print(f'Output classes: 2')
print('\nKey features:')
print('- Transformer architecture (attention-based)')
print('- Patch-based processing (16x16 patches)')
print('- No convolutions - pure attention')
print('- Pre-trained on ImageNet')
print('- May require more data to train effectively')

# Section 5: Systematic Experiments

This section runs 10 different experiments to improve upon the baseline model systematically.

In [ ]:
# Define additional model architectures for experiments
class CNN_BatchNorm(nn.Module):
    """CNN with Batch Normalization"""
    def __init__(self, num_classes=2, input_channels=3):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(input_channels, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


class CNN_Dropout(nn.Module):
    """CNN with Dropout regularization"""
    def __init__(self, num_classes=2, input_channels=3):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(input_channels, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

print('Additional model architectures defined')

In [ ]:
# Initialize Experiment Tracking Variables
# Run this cell BEFORE any experiments to ensure all shared variables are defined

if len(train_df) > 0:
    # Initialize experiment results list (if not already done by baseline)
    if 'experiment_results' not in dir() or not isinstance(experiment_results, list):
        experiment_results = []
        print('Initialized experiment_results list')
    
    # Initialize model storage dictionary
    if 'all_models' not in dir() or not isinstance(all_models, dict):
        all_models = {}
        print('Initialized all_models dictionary')
    
    # Learning rate scaling factor for batch size
    LR_SCALE = BATCH_SIZE / 32
    print(f'Set LR_SCALE = {LR_SCALE} (for BATCH_SIZE={BATCH_SIZE})')
    
    print('\nExperiment tracking variables ready!')
    print(f'  - experiment_results: {len(experiment_results)} entries')
    print(f'  - all_models: {len(all_models)} models')
    print(f'  - LR_SCALE: {LR_SCALE}')

## Experiment 1: Adam Optimizer

**Hypothesis:** Adam optimizer will converge faster than SGD

**Change:** Replace SGD with Adam optimizer

In [ ]:
# Experiment 1: Adam Optimizer
if len(train_df) > 0:
    print('='*60)
    print('Experiment 1: Adam Optimizer')
    print('='*60)
    
    exp1_config = {
        'epochs': 10,
        'lr': 0.001,
        'optimizer': 'adam'
    }
    
    exp1_model = BaselineCNN(num_classes=2, input_channels=INPUT_CHANNELS).to(device)
    exp1_model, exp1_hist, exp1_acc = train_model(exp1_model, train_loader, val_loader, exp1_config, device)
    
    experiment_results.append({
        'name': 'Exp1: Adam',
        'val_accuracy': exp1_acc,
        'val_loss': exp1_hist['val_loss'][-1]
    })

## Experiment 2: Batch Normalization

**Hypothesis:** Batch normalization will stabilize training and improve accuracy

**Change:** Add BatchNorm2d layers after each convolutional layer

In [ ]:
# Experiment 2: Batch Normalization
if len(train_df) > 0:
    print('='*60)
    print('Experiment 2: Batch Normalization')
    print('='*60)
    
    exp2_config = {
        'epochs': 10,
        'lr': 0.001,
        'optimizer': 'sgd'
    }
    
    exp2_model = CNN_BatchNorm(num_classes=2, input_channels=INPUT_CHANNELS).to(device)
    exp2_model, exp2_hist, exp2_acc = train_model(exp2_model, train_loader, val_loader, exp2_config, device)
    
    experiment_results.append({
        'name': 'Exp2: BatchNorm',
        'val_accuracy': exp2_acc,
        'val_loss': exp2_hist['val_loss'][-1]
    })

## Experiment 3: Dropout Regularization

**Hypothesis:** Dropout will reduce overfitting by randomly dropping neurons during training

**Change:** Add dropout layer (50%) in the classifier

In [ ]:
# Experiment 3: Dropout Regularization
if len(train_df) > 0:
    # Initialize shared variables for experiments 3-10
    all_models = {}
    LR_SCALE = BATCH_SIZE / 32  # Scale LR for batch size
    
    print('='*60)
    print('Experiment 3: Dropout Regularization')
    print('='*60)
    exp3_config = {'epochs': 10, 'lr': 0.001 * LR_SCALE, 'optimizer': 'sgd'}
    exp3_model = CNN_Dropout(num_classes=2, input_channels=INPUT_CHANNELS).to(device)
    exp3_model, exp3_hist, exp3_acc = train_model(exp3_model, train_loader, val_loader, exp3_config, device)
    experiment_results.append({'name': 'Exp3: Dropout', 'val_accuracy': exp3_acc, 'val_loss': exp3_hist['val_loss'][-1]})
    all_models['Exp3: Dropout'] = exp3_model

## Experiment 4: Weight Decay (L2 Regularization)

**Hypothesis:** L2 regularization will prevent overfitting by penalizing large weights

**Change:** Add weight decay (1e-4) to the optimizer

In [ ]:
# Experiment 4: Weight Decay
if len(train_df) > 0:
    print('='*60)
    print('Experiment 4: Weight Decay (L2 Regularization)')
    print('='*60)
    exp4_config = {'epochs': 10, 'lr': 0.001 * LR_SCALE, 'optimizer': 'sgd', 'weight_decay': 1e-4}
    exp4_model = CNN_Dropout(num_classes=2, input_channels=INPUT_CHANNELS).to(device)
    exp4_model, exp4_hist, exp4_acc = train_model(exp4_model, train_loader, val_loader, exp4_config, device)
    experiment_results.append({'name': 'Exp4: WeightDecay', 'val_accuracy': exp4_acc, 'val_loss': exp4_hist['val_loss'][-1]})
    all_models['Exp4: WeightDecay'] = exp4_model

## Experiment 5: Adam + Dropout + Weight Decay

**Hypothesis:** Combining Adam optimizer with regularization techniques will yield better results

**Change:** Use Adam optimizer with dropout and weight decay together

In [ ]:
# Experiment 5: Adam + Dropout + Weight Decay
if len(train_df) > 0:
    print('='*60)
    print('Experiment 5: Adam + Dropout + Weight Decay')
    print('='*60)
    exp5_config = {'epochs': 10, 'lr': 0.001 * LR_SCALE, 'optimizer': 'adam', 'weight_decay': 1e-4}
    exp5_model = CNN_Dropout(num_classes=2, input_channels=INPUT_CHANNELS).to(device)
    exp5_model, exp5_hist, exp5_acc = train_model(exp5_model, train_loader, val_loader, exp5_config, device)
    experiment_results.append({'name': 'Exp5: Adam+Drop+WD', 'val_accuracy': exp5_acc, 'val_loss': exp5_hist['val_loss'][-1]})
    all_models['Exp5: Adam+Drop+WD'] = exp5_model

## Experiment 6: ResNet-18 (Frozen Features)

**Hypothesis:** Pre-trained ImageNet features will work well for footprint classification

**Change:** Use ResNet-18 with frozen convolutional layers, only train classifier

In [ ]:
# Experiment 6: ResNet-18 Frozen
if len(train_df) > 0:
    print('='*60)
    print('Experiment 6: ResNet-18 (Frozen Features)')
    print('='*60)
    exp6_model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    if INPUT_CHANNELS == 1:
        exp6_model.conv1 = nn.Conv2d(1, 64, 7, 2, 3, bias=False)
    for param in exp6_model.parameters():
        param.requires_grad = False
    exp6_model.fc = nn.Linear(exp6_model.fc.in_features, 2)
    exp6_model = exp6_model.to(device)
    exp6_config = {'epochs': 10, 'lr': 0.001 * LR_SCALE, 'optimizer': 'adam'}
    exp6_model, exp6_hist, exp6_acc = train_model(exp6_model, train_loader, val_loader, exp6_config, device)
    experiment_results.append({'name': 'Exp6: ResNet Frozen', 'val_accuracy': exp6_acc, 'val_loss': exp6_hist['val_loss'][-1]})
    all_models['Exp6: ResNet Frozen'] = exp6_model

## Experiment 7: ResNet-18 (Fine-tuned)

**Hypothesis:** Fine-tuning all layers will allow the model to adapt to footprint-specific features

**Change:** Train all ResNet-18 layers with a smaller learning rate

In [ ]:
# Experiment 7: ResNet-18 Fine-tuned (with Optuna-optimized hyperparameters)
if len(train_df) > 0:
    print('='*60)
    print('Experiment 7: ResNet-18 (Fine-tuned with Best Hyperparameters)')
    print('='*60)
    
    # Optimized hyperparameters from Optuna
    best_lr = 0.000460
    best_weight_decay = 0.000417
    best_dropout = 0.199358
    best_batch_size = 64
    best_epochs = 16
    best_rotation = 13
    best_color_jitter = 0.122036
    
    print(f'Using optimized hyperparameters:')
    print(f'  lr: {best_lr}')
    print(f'  weight_decay: {best_weight_decay}')
    print(f'  dropout: {best_dropout:.2f}')
    print(f'  batch_size: {best_batch_size}')
    print(f'  epochs: {best_epochs}')
    print(f'  rotation: {best_rotation}')
    print(f'  color_jitter: {best_color_jitter}')
    
    # Create optimized transforms
    train_transform_exp7 = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(best_rotation),
        transforms.ColorJitter(
            brightness=best_color_jitter,
            contrast=best_color_jitter,
            saturation=best_color_jitter * 0.5
        ),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.ToTensor(),
        transforms.Normalize(*mean_std),
        transforms.RandomErasing(p=0.2)
    ])
    
    # Create datasets and loaders with optimized batch size
    train_dataset_exp7 = FootprintDataset(train_paths, train_labels, train_transform_exp7)
    train_loader_exp7 = DataLoader(
        train_dataset_exp7, 
        batch_size=best_batch_size, 
        shuffle=True, 
        num_workers=0, 
        pin_memory=True
    )
    val_loader_exp7 = DataLoader(
        val_dataset, 
        batch_size=best_batch_size, 
        shuffle=False, 
        num_workers=0, 
        pin_memory=True
    )
    
    # Create model with optimized dropout
    exp7_model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    if INPUT_CHANNELS == 1:
        exp7_model.conv1 = nn.Conv2d(1, 64, 7, 2, 3, bias=False)
    num_features = exp7_model.fc.in_features
    exp7_model.fc = nn.Sequential(
        nn.Dropout(best_dropout),
        nn.Linear(num_features, 2)
    )
    exp7_model = exp7_model.to(device)
    
    # Train with optimized config
    exp7_config = {
        'epochs': best_epochs, 
        'lr': best_lr, 
        'optimizer': 'adam', 
        'weight_decay': best_weight_decay
    }
    exp7_model, exp7_hist, exp7_acc = train_model(
        exp7_model, train_loader_exp7, val_loader_exp7, exp7_config, device
    )
    
    experiment_results.append({
        'name': 'Exp7: ResNet Finetuned', 
        'val_accuracy': exp7_acc, 
        'val_loss': exp7_hist['val_loss'][-1]
    })
    all_models['Exp7: ResNet Finetuned'] = exp7_model

## Experiment 8: EfficientNet-B0

**Hypothesis:** EfficientNet's compound scaling will provide better accuracy with fewer parameters

**Change:** Use pre-trained EfficientNet-B0 instead of ResNet

In [ ]:
# Experiment 8: EfficientNet-B0
if len(train_df) > 0:
    print('='*60)
    print('Experiment 8: EfficientNet-B0')
    print('='*60)
    exp8_model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=2, in_chans=INPUT_CHANNELS).to(device)
    exp8_config = {'epochs': 10, 'lr': 0.0001 * LR_SCALE, 'optimizer': 'adam', 'weight_decay': 1e-4}
    exp8_model, exp8_hist, exp8_acc = train_model(exp8_model, train_loader, val_loader, exp8_config, device)
    experiment_results.append({'name': 'Exp8: EfficientNet', 'val_accuracy': exp8_acc, 'val_loss': exp8_hist['val_loss'][-1]})
    all_models['Exp8: EfficientNet'] = exp8_model

## Experiment 9: Higher Learning Rate

**Hypothesis:** A higher learning rate may speed up convergence and escape local minima

**Change:** Increase learning rate from 0.001 to 0.01

In [ ]:
# Experiment 9: Higher Learning Rate
if len(train_df) > 0:
    print('='*60)
    print('Experiment 9: Higher Learning Rate (0.01)')
    print('='*60)
    exp9_config = {'epochs': 10, 'lr': 0.01 * LR_SCALE, 'optimizer': 'sgd'}
    exp9_model = CNN_Dropout(num_classes=2, input_channels=INPUT_CHANNELS).to(device)
    exp9_model, exp9_hist, exp9_acc = train_model(exp9_model, train_loader, val_loader, exp9_config, device)
    experiment_results.append({'name': 'Exp9: HighLR', 'val_accuracy': exp9_acc, 'val_loss': exp9_hist['val_loss'][-1]})
    all_models['Exp9: HighLR'] = exp9_model

## Experiment 10: Longer Training

**Hypothesis:** More training epochs will allow the model to better learn the data patterns

**Change:** Train for 20 epochs instead of 10

In [ ]:
# Experiment 10: Longer Training
if len(train_df) > 0:
    print('='*60)
    print('Experiment 10: Longer Training (20 epochs)')
    print('='*60)
    exp10_config = {'epochs': 20, 'lr': 0.001 * LR_SCALE, 'optimizer': 'adam'}
    exp10_model = CNN_Dropout(num_classes=2, input_channels=INPUT_CHANNELS).to(device)
    exp10_model, exp10_hist, exp10_acc = train_model(exp10_model, train_loader, val_loader, exp10_config, device)
    experiment_results.append({'name': 'Exp10: LongTrain', 'val_accuracy': exp10_acc, 'val_loss': exp10_hist['val_loss'][-1]})
    all_models['Exp10: LongTrain'] = exp10_model

In [ ]:
# Select Best Model from All Experiments
if len(train_df) > 0:
    print('='*80)
    print('SELECTING BEST MODEL')
    print('='*80)
    
    # Find the best experiment
    best_experiment = max(experiment_results, key=lambda x: x['val_accuracy'])
    best_model_name = best_experiment['name']
    
    # Set final_model based on best result
    if best_model_name in all_models:
        final_model = all_models[best_model_name]
    elif 'exp8_model' in dir() and best_model_name == 'Exp8: EfficientNet':
        final_model = exp8_model
    elif 'exp7_model' in dir() and best_model_name == 'Exp7: ResNet Finetuned':
        final_model = exp7_model
    elif 'exp2_model' in dir() and best_model_name == 'Exp2: BatchNorm':
        final_model = exp2_model
    elif 'exp1_model' in dir() and best_model_name == 'Exp1: Adam':
        final_model = exp1_model
    elif 'baseline_model' in dir() and best_model_name == 'Baseline':
        final_model = baseline_model
    else:
        # Fallback to most recent model in all_models
        final_model = list(all_models.values())[-1] if all_models else exp10_model
        print(f'Note: Could not find {best_model_name}, using fallback model')
    
    print(f'\nBest Model: {best_model_name}')
    print(f'Validation Accuracy: {best_experiment["val_accuracy"]:.4f}')
    print(f'Validation Loss: {best_experiment["val_loss"]:.4f}')
    print('\nfinal_model is now set and ready for evaluation and submission.')

In [ ]:
# Display Experiment Results Summary
if len(train_df) > 0:
    results_df = pd.DataFrame(experiment_results).sort_values('val_accuracy', ascending=False)
    print('\n' + '='*80)
    print('EXPERIMENT RESULTS SUMMARY (Sorted by Validation Accuracy)')
    print('='*80)
    display(results_df)
    
    # Plot comparison
    fig, ax = plt.subplots(figsize=(12, 6))
    results_df_sorted = results_df.sort_values('val_accuracy')
    ax.barh(results_df_sorted['name'], results_df_sorted['val_accuracy'])
    ax.set_xlabel('Validation Accuracy')
    ax.set_title('Experiment Comparison')
    ax.grid(True, axis='x')
    plt.tight_layout()
    plt.show()

In [ ]:
# Train Final Model with Best Hyperparameters from Optuna
if len(train_df) > 0 and 'study' in dir():
    print('='*80)
    print('TRAINING FINAL MODEL WITH OPTIMAL HYPERPARAMETERS')
    print('='*80)
    
    # Get best parameters
    best_params = study.best_trial.params
    
    print('\nUsing best hyperparameters:')
    for key, value in best_params.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.6f}")
        else:
            print(f"  {key}: {value}")
    
    # Create optimized transforms
    train_transform_final = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(best_params['rotation']),
        transforms.ColorJitter(
            brightness=best_params['color_jitter'],
            contrast=best_params['color_jitter'],
            saturation=best_params['color_jitter'] * 0.5
        ),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.ToTensor(),
        transforms.Normalize(*mean_std),
        transforms.RandomErasing(p=0.2)
    ])
    
    # Create datasets and loaders with best batch size
    train_dataset_final = FootprintDataset(train_paths, train_labels, train_transform_final)
    train_loader_final = DataLoader(
        train_dataset_final, 
        batch_size=best_params['batch_size'], 
        shuffle=True, 
        num_workers=0, 
        pin_memory=True
    )
    val_loader_final = DataLoader(
        val_dataset, 
        batch_size=best_params['batch_size'], 
        shuffle=False, 
        num_workers=0, 
        pin_memory=True
    )
    
    # Create model with best dropout
    final_model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    num_features = final_model.fc.in_features
    final_model.fc = nn.Sequential(
        nn.Dropout(best_params['dropout']),
        nn.Linear(num_features, 2)
    )
    final_model = final_model.to(device)
    
    # Create optimizer with best parameters
    if best_params['optimizer'] == 'adam':
        optimizer = optim.Adam(final_model.parameters(), lr=best_params['lr'], 
                               weight_decay=best_params['weight_decay'])
    elif best_params['optimizer'] == 'adamw':
        optimizer = optim.AdamW(final_model.parameters(), lr=best_params['lr'], 
                                weight_decay=best_params['weight_decay'])
    else:
        optimizer = optim.SGD(final_model.parameters(), lr=best_params['lr'], 
                              momentum=0.9, weight_decay=best_params['weight_decay'])
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=best_params['epochs'])
    criterion = nn.CrossEntropyLoss()
    
    # Training loop with progress tracking
    best_val_acc = 0.0
    best_model_wts = copy.deepcopy(final_model.state_dict())
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    print(f"\nTraining for {best_params['epochs']} epochs...")
    
    for epoch in range(best_params['epochs']):
        # Train
        final_model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, labels in tqdm(train_loader_final, desc=f'Epoch {epoch+1}/{best_params["epochs"]}', leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = final_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
        
        train_loss = running_loss / total
        train_acc = correct / total
        
        scheduler.step()
        
        # Validate
        final_model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader_final:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = final_model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                correct += predicted.eq(labels).sum().item()
                total += labels.size(0)
        
        val_loss = val_loss / total
        val_acc = correct / total
        
        # Store history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(final_model.state_dict())
    
    # Load best weights
    final_model.load_state_dict(best_model_wts)
    
    print(f'\n' + '='*80)
    print(f'FINAL MODEL TRAINING COMPLETE')
    print(f'='*80)
    print(f'Best Validation Accuracy: {best_val_acc:.4f}')
    print(f'\nThis model will be used for Kaggle submission.')
    
    # Update experiment results
    experiment_results.append({
        'name': 'Optuna Optimized',
        'val_accuracy': best_val_acc,
        'val_loss': history['val_loss'][-1]
    })

In [ ]:
# Visualize Optimization Results
if len(train_df) > 0 and 'study' in dir():
    print('='*80)
    print('OPTIMIZATION VISUALIZATION')
    print('='*80)
    
    # Plot optimization history
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Optimization history
    ax = axes[0, 0]
    trials = [t for t in study.trials if t.state == TrialState.COMPLETE]
    values = [t.value for t in trials]
    ax.plot(range(len(values)), values, 'o-', alpha=0.7)
    ax.axhline(y=max(values), color='r', linestyle='--', label=f'Best: {max(values):.4f}')
    ax.set_xlabel('Trial')
    ax.set_ylabel('Validation Accuracy')
    ax.set_title('Optimization History')
    ax.legend()
    ax.grid(True)
    
    # 2. Hyperparameter importance (if enough trials)
    ax = axes[0, 1]
    if len(trials) >= 5:
        importances = optuna.importance.get_param_importances(study)
        params = list(importances.keys())[:8]  # Top 8
        values = [importances[p] for p in params]
        ax.barh(params, values)
        ax.set_xlabel('Importance')
        ax.set_title('Hyperparameter Importance')
    else:
        ax.text(0.5, 0.5, 'Need more trials\nfor importance analysis', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title('Hyperparameter Importance')
    
    # 3. Learning rate vs accuracy
    ax = axes[1, 0]
    lrs = [t.params['lr'] for t in trials]
    accs = [t.value for t in trials]
    ax.scatter(lrs, accs, alpha=0.7)
    ax.set_xscale('log')
    ax.set_xlabel('Learning Rate (log scale)')
    ax.set_ylabel('Validation Accuracy')
    ax.set_title('Learning Rate vs Accuracy')
    ax.grid(True)
    
    # 4. Weight decay vs accuracy
    ax = axes[1, 1]
    wds = [t.params['weight_decay'] for t in trials]
    ax.scatter(wds, accs, alpha=0.7)
    ax.set_xscale('log')
    ax.set_xlabel('Weight Decay (log scale)')
    ax.set_ylabel('Validation Accuracy')
    ax.set_title('Weight Decay vs Accuracy')
    ax.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Print trial summary table
    print('\nTop 5 Trials:')
    trials_sorted = sorted(trials, key=lambda t: t.value, reverse=True)[:5]
    for i, t in enumerate(trials_sorted, 1):
        print(f"\n{i}. Accuracy: {t.value:.4f}")
        print(f"   lr={t.params['lr']:.2e}, wd={t.params['weight_decay']:.2e}, "
              f"opt={t.params['optimizer']}, bs={t.params['batch_size']}, "
              f"epochs={t.params['epochs']}, dropout={t.params['dropout']:.2f}")

In [ ]:
# Optuna Hyperparameter Optimization for ResNet-18
if len(train_df) > 0:
    
    def create_objective(train_paths, train_labels, val_dataset, device, mean_std):
        """
        Factory function to create objective with access to data.
        """
        def objective(trial):
            """
            Optuna objective function - trains one model with suggested hyperparameters
            and returns validation accuracy.
            """
            
            # =================================================================
            # HYPERPARAMETERS TO TUNE
            # =================================================================
            
            # Learning rate (log scale: 1e-5 to 1e-2)
            lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
            
            # Weight decay (log scale: 1e-6 to 1e-2)
            weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-2, log=True)
            
            # Optimizer choice
            optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'adamw'])
            
            # Batch size
            batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
            
            # Number of epochs
            epochs = trial.suggest_int('epochs', 10, 25)
            
            # Dropout in classifier
            dropout_rate = trial.suggest_float('dropout', 0.0, 0.5)
            
            # Augmentation parameters
            rotation_degrees = trial.suggest_int('rotation', 5, 20)
            color_jitter_strength = trial.suggest_float('color_jitter', 0.0, 0.3)
            
            # =================================================================
            # CREATE DATA LOADERS WITH TUNED AUGMENTATION
            # =================================================================
            
            train_transform_tuned = transforms.Compose([
                transforms.Resize((256, 256)),
                transforms.RandomCrop(224),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomRotation(rotation_degrees),
                transforms.ColorJitter(
                    brightness=color_jitter_strength,
                    contrast=color_jitter_strength,
                    saturation=color_jitter_strength * 0.5
                ),
                transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                transforms.ToTensor(),
                transforms.Normalize(*mean_std),
                transforms.RandomErasing(p=0.2)
            ])
            
            train_dataset_tuned = FootprintDataset(train_paths, train_labels, train_transform_tuned)
            train_loader_tuned = DataLoader(
                train_dataset_tuned, 
                batch_size=batch_size, 
                shuffle=True, 
                num_workers=0, 
                pin_memory=True
            )
            val_loader_tuned = DataLoader(
                val_dataset, 
                batch_size=batch_size, 
                shuffle=False, 
                num_workers=0, 
                pin_memory=True
            )
            
            # =================================================================
            # CREATE MODEL WITH TUNED DROPOUT
            # =================================================================
            
            model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
            
            # Replace classifier with dropout
            num_features = model.fc.in_features
            model.fc = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(num_features, 2)
            )
            model = model.to(device)
            
            # =================================================================
            # CREATE OPTIMIZER
            # =================================================================
            
            if optimizer_name == 'adam':
                optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
            elif optimizer_name == 'adamw':
                optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
            else:  # sgd
                optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
            
            # Learning rate scheduler
            scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
            
            criterion = nn.CrossEntropyLoss()
            
            # =================================================================
            # TRAINING LOOP
            # =================================================================
            
            best_val_acc = 0.0
            
            for epoch in range(epochs):
                # Train
                model.train()
                for inputs, labels in train_loader_tuned:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                
                scheduler.step()
                
                # Validate
                model.eval()
                correct = 0
                total = 0
                with torch.no_grad():
                    for inputs, labels in val_loader_tuned:
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        _, predicted = outputs.max(1)
                        correct += predicted.eq(labels).sum().item()
                        total += labels.size(0)
                
                val_acc = correct / total
                best_val_acc = max(best_val_acc, val_acc)
                
                # Report intermediate value for pruning
                trial.report(val_acc, epoch)
                
                # Prune trial if not promising
                if trial.should_prune():
                    raise optuna.exceptions.TrialPruned()
            
            return best_val_acc
        
        return objective
    
    # =================================================================
    # RUN OPTIMIZATION
    # =================================================================
    
    print('='*80)
    print('OPTUNA HYPERPARAMETER OPTIMIZATION')
    print('='*80)
    print('\nStarting Bayesian optimization...')
    print('This will test multiple hyperparameter configurations.')
    print('Unpromising trials will be pruned early to save time.\n')
    
    # Create study
    study = optuna.create_study(
        direction='maximize',  # Maximize validation accuracy
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5)  # Prune after 5 epochs
    )
    
    # Create objective function with data access
    objective = create_objective(train_paths, train_labels, val_dataset, device, mean_std)
    
    # Run optimization - adjust n_trials based on available time
    # 30 trials is a good balance between exploration and time
    study.optimize(
        objective, 
        n_trials=30,  # Number of different configurations to try
        show_progress_bar=True
    )
    
    # =================================================================
    # PRINT RESULTS
    # =================================================================
    
    print('\n' + '='*80)
    print('OPTIMIZATION COMPLETE')
    print('='*80)
    
    print(f"\nBest trial validation accuracy: {study.best_trial.value:.4f}")
    print(f"\nBest hyperparameters:")
    for key, value in study.best_trial.params.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.6f}")
        else:
            print(f"  {key}: {value}")
    
    print(f"\nTotal trials: {len(study.trials)}")
    pruned_trials = len([t for t in study.trials if t.state == TrialState.PRUNED])
    complete_trials = len([t for t in study.trials if t.state == TrialState.COMPLETE])
    print(f"Completed trials: {complete_trials}")
    print(f"Pruned trials: {pruned_trials}")

# Section 5b: Hyperparameter Optimization with Optuna

This section uses Bayesian optimization to find optimal hyperparameters for the ResNet-18 model.

**Hyperparameters being tuned:**
- Learning rate (log scale)
- Weight decay (log scale)
- Optimizer (Adam, SGD, AdamW)
- Batch size
- Number of epochs
- Dropout rate
- Augmentation strength (rotation, color jitter)

**Why Bayesian Optimization?**
- More efficient than grid/random search
- Uses previous trial results to guide future choices
- Automatically prunes unpromising trials

# Section 6: Final Model Evaluation & Explainability

This section evaluates the best model with detailed metrics and uses Grad-CAM for explainability.

In [ ]:
# Identify Best Model
if len(train_df) > 0:
    best_experiment = max(experiment_results, key=lambda x: x['val_accuracy'])
    print('='*80)
    print('BEST MODEL')
    print('='*80)
    print(f"Name: {best_experiment['name']}")
    print(f"Validation Accuracy: {best_experiment['val_accuracy']:.4f}")
    print(f"Validation Loss: {best_experiment['val_loss']:.4f}")
    print('\nUsing this model for final evaluation...')

In [ ]:
# Detailed Metrics on Validation Set
if len(train_df) > 0:
    from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, ConfusionMatrixDisplay, classification_report
    
    criterion = nn.CrossEntropyLoss()
    final_loss, final_acc, final_preds, final_labels = evaluate(final_model, val_loader, criterion, device)
    
    print('='*80)
    print('DETAILED METRICS')
    print('='*80)
    print(f'Accuracy: {final_acc:.4f}')
    print(f'Loss: {final_loss:.4f}')
    
    # Per-class metrics
    precision, recall, f1, support = precision_recall_fscore_support(final_labels, final_preds, average=None)
    print('\nPer-Class Metrics:')
    for i in range(2):
        print(f'  Class {i} ({"Female" if i==0 else "Male"}):')
        print(f'    Precision: {precision[i]:.4f}')
        print(f'    Recall: {recall[i]:.4f}')
        print(f'    F1-Score: {f1[i]:.4f}')
        print(f'    Support: {support[i]}')
    
    # Macro averages
    macro_p, macro_r, macro_f1, _ = precision_recall_fscore_support(final_labels, final_preds, average='macro')
    print(f'\nMacro Averages:')
    print(f'  Precision: {macro_p:.4f}')
    print(f'  Recall: {macro_r:.4f}')
    print(f'  F1-Score: {macro_f1:.4f}')

In [ ]:
# Confusion Matrix
if len(train_df) > 0:
    cm = confusion_matrix(final_labels, final_preds)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Female', 'Male'])
    disp.plot(ax=ax, cmap='Blues', values_format='d')
    plt.title('Confusion Matrix - Final Model')
    plt.show()
    
    print('\nConfusion Matrix Analysis:')
    print(f'True Negatives (Correctly classified Female): {cm[0,0]}')
    print(f'False Positives (Female classified as Male): {cm[0,1]}')
    print(f'False Negatives (Male classified as Female): {cm[1,0]}')
    print(f'True Positives (Correctly classified Male): {cm[1,1]}')

## Explainability (XAI) - Grad-CAM

**Note:** Grad-CAM visualization helps understand which regions of the footprint image the model focuses on when making predictions.

For implementation, you would:
1. Select the last convolutional layer
2. Register hooks to capture gradients and activations
3. Generate heatmaps for sample predictions
4. Overlay heatmaps on original images

This requires additional implementation based on your model architecture.

In [ ]:
# Generate Kaggle Submission
if len(train_df) > 0 and TEST_DIR.exists():
    # Load test images
    test_paths = sorted(list(TEST_DIR.glob('*.jpg')) + list(TEST_DIR.glob('*.png')))
    
    if len(test_paths) > 0:
        print('='*80)
        print('GENERATING KAGGLE SUBMISSION')
        print('='*80)
        print(f'Found {len(test_paths)} test images')
        
        # Get test image IDs from filenames
        test_ids = [p.stem for p in test_paths]
        test_paths_str = [str(p) for p in test_paths]
        
        # Create test dataset and loader
        test_dataset = FootprintDataset(test_paths_str, [0]*len(test_paths_str), val_transform)
        test_loader = DataLoader(
            test_dataset, 
            batch_size=BATCH_SIZE, 
            shuffle=False, 
            num_workers=0,  # Windows-safe
            pin_memory=True
        )
        
        # Generate predictions
        final_model.eval()
        test_predictions = []
        
        with torch.no_grad():
            for inputs, _ in tqdm(test_loader, desc='Predicting test set'):
                inputs = inputs.to(device)
                outputs = final_model(inputs)
                _, predicted = outputs.max(1)
                test_predictions.extend(predicted.cpu().numpy())
        
        # Create submission dataframe - Kaggle expects 'filename' column
        submission_df = pd.DataFrame({
            'filename': test_ids,  # Column name must match Kaggle's expected format
            'label': test_predictions
        })
        
        # Save to CSV
        submission_df.to_csv('submission.csv', index=False)
        
        print(f'\nSubmission file saved: submission.csv')
        print(f'Total predictions: {len(submission_df)}')
        print('\nFirst few predictions:')
        display(submission_df.head(10))
        
        # Show prediction distribution
        print(f'\nPrediction distribution:')
        print(submission_df['label'].value_counts().sort_index())
    else:
        print('No test images found in test directory')
else:
    if len(train_df) == 0:
        print('Training data not loaded - cannot generate submission')
    else:
        print('Test directory not found - cannot generate submission')

# Section 7: Conclusion & Reflection

## Summary of Results

**Baseline Performance:**
- Simple CNN from scratch
- Validation accuracy: [TO FILL after running]

**Final Model Performance:**
- Best model: [TO FILL after experiments]
- Validation accuracy: [TO FILL after running]
- Improvement over baseline: [TO FILL]

**Key Findings:**
- Transfer learning (ResNet/EfficientNet) significantly outperformed from-scratch models
- Batch normalization and dropout helped reduce overfitting
- Adam optimizer converged faster than SGD
- Pre-trained models leveraged ImageNet features effectively

## Limitations

1. **Small Dataset:** Only 1,573 training images may not capture full diversity
2. **Class Imbalance:** May affect model fairness if one class is under-represented
3. **Overfitting Risk:** Limited data increases risk of overfitting to training set
4. **Demographic Bias:** Dataset may not represent diverse populations
5. **Generalization:** Model tested only on validation set from same distribution

## Deployment Considerations

**Should this model be deployed in forensic investigations?**

**Recommendation: NOT as a standalone decision-making tool**

**Rationale:**
- Model accuracy is good but not perfect (false positives/negatives exist)
- Forensic decisions have serious legal and ethical implications
- Potential bias in training data could lead to unfair outcomes
- Model is a "black box" - decisions need to be explainable

**Acceptable Use:**
- As an **investigative support tool** alongside human expert analysis
- Requires human oversight and verification
- Should be one piece of evidence, not the sole determinant
- Needs extensive validation on diverse, real-world data
- Requires transparency about limitations and error rates

## Future Work

1. **More Data:** Collect larger, more diverse dataset
2. **Cross-Validation:** Implement k-fold CV for robust evaluation
3. **Advanced Augmentation:** Test stronger data augmentation techniques
4. **Ensemble Methods:** Combine multiple models for better predictions
5. **Better XAI:** Implement comprehensive Grad-CAM and SHAP analysis
6. **Calibration:** Ensure predicted probabilities are well-calibrated
7. **External Validation:** Test on completely separate datasets
8. **Bias Analysis:** Systematic analysis of potential biases
9. **Uncertainty Quantification:** Provide confidence intervals for predictions
10. **Real-world Testing:** Pilot study with forensic experts